In [0]:
%python
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, current_date, expr
from pyspark.sql.window import Window

# Initialize Spark Session
spark = SparkSession.builder.appName("SCD2_Dimension_Load").getOrCreate()

# Sample Source Data
df_source = spark.createDataFrame([
    (1, "Alice", "HR", "2024-02-01"),
    (2, "Bob", "IT", "2024-02-01"),
    (3, "Charlie", "Finance", "2024-02-01"),
    (4, "DAVID", "HR", "2024-02-01"),
], ["id", "name", "department", "effective_date"])

# Sample Target Data (Existing Dimension Table)
df_target = spark.createDataFrame([
    (1, "Alice", "Finance", "2023-01-01", "9999-12-31", "Y"),
    (2, "Bob", "IT", "2023-06-01", "9999-12-31", "Y"),
    (3, "Charlie", "Finance", "2023-07-01", "9999-12-31", "Y"),
], ["id", "name", "department", "start_date", "end_date", "current_flag"])

df_source.show()
df_target.show()

# Step 1: Identify new and changed records
joined_df = df_source.alias("src").join(
    df_target.alias("tgt"), "id", "left_outer"
).select(
    col("src.id"), col("src.name"), col("src.department"), col("src.effective_date"),
    col("tgt.id").alias("existing_id"),col("tgt.department"),
    col("tgt.start_date"), col("tgt.end_date"), col("tgt.current_flag")
)
joined_df.show()

# Step 2: Filter records that have changed -- (insert records)
changed_records = joined_df.filter(
    (col("existing_id").isNotNull()) & (col("src.department") != col("tgt.department")) 
)

# Step 3: Close out old records (inner join but display only the records from left table)
updated_target = df_target.join(
    changed_records.select("id"), "id", "left_semi"
).withColumn("end_date", current_date()).withColumn("current_flag", lit("N"))


# Step 4: Insert new versions of changed records
new_versions = changed_records.withColumn("start_date", col("effective_date"))\
                .withColumn("end_date", lit("9999-12-31"))\
                .withColumn("current_flag", lit("Y"))
new_versions.show()

# Step 5: Identify new records
new_records = joined_df.filter(col("existing_id").isNull())\
                .withColumn("start_date", col("effective_date"))\
                .withColumn("end_date", lit("9999-12-31"))\
                .withColumn("current_flag", lit("Y"))
new_records.show()

# Step 6: Extract unchanged records
unchanged_records = df_target.join(
    changed_records.select("id"), "id", "left_anti"
).join(
    new_records.select("id"), "id", "left_anti"
)
#selecting the records that is not in both changed_records and new_records using anti join
unchanged_records.show()

# Step 7: Prepare final dimension table
final_target = updated_target.union(new_versions.select("id", "name", "src.department", "start_date", "end_date", "current_flag"))#upd, upd_ins
final_target = final_target.union(new_records.select("id", "name", "src.department", "start_date", "end_date", "current_flag"))#upd, upd_ins,ins
final_target = final_target.union(unchanged_records)#no change records
# Show final output
final_target.show() #final target